In [27]:
#Dependencies and Setup

import pandas as pd
import numpy as np
from statistics import mean
#import statistics

fantasy_df = pd.read_csv('Resources/purchase_data.csv')
#rename to sales_data

# Player Count

In [28]:
player_count = pd.DataFrame([{'Total Players': 0}])
num_of_players = len(fantasy_df['SN'].unique())
player_count['Total Players'] = num_of_players
player_count

,Total Players
0,576


# Purchasing Analysis (Total)

In [34]:
purchase_analysis = pd.DataFrame(np.array([[0,0,0,0]]), columns=['Number of Unique Items', 'Average Price', 
                                          'Number of Purchases', 'Total Revenue'])
inventory_df = pd.DataFrame(fantasy_df[['Item ID','Item Name', 'Price']].drop_duplicates(['Item ID']))
purchase_analysis['Number of Unique Items'] = len(inventory_df)
purchase_analysis['Average Price'] = mean(list(inventory_df['Price']))
purchase_analysis['Average Price'] = purchase_analysis['Average Price'].map("${:,.2f}".format)
purchase_analysis['Number of Purchases'] = len(fantasy_df['Item ID'])
purchase_analysis['Total Revenue'] = fantasy_df['Price'].sum()
purchase_analysis['Total Revenue'] = purchase_analysis['Total Revenue'].map("${:,.2f}".format)
purchase_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.04,780,"$2,379.77"


# Gender Demographics

In [46]:
gender_demo = fantasy_df.drop_duplicates(['SN'])
gender_demo = pd.DataFrame(gender_demo['Gender'].value_counts())
gender_demo = gender_demo.rename(columns={'Gender': 'Total Count'})
gender_demo['Percentage of Players (%)'] = round((gender_demo['Total Count']/num_of_players)*100,2)
gender_demo

,Total Count,Percentage of Players (%)
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


# Purchasing Analysis (Gender)

In [73]:
purchase_analysis = fantasy_df.groupby('Gender').agg({'Item ID': 'count', 'Price': ['sum', lambda x: round(x.unique().mean(),2)]})
purchase_analysis.columns = purchase_analysis.columns.droplevel(level=1)

#average purchase by person by gender
purchase_analysis2 = fantasy_df.groupby(['Gender', 'SN']).agg({'Item ID': 'count', 'Price': 'sum'})
purchase_analysis2 = purchase_analysis2.groupby('Gender').agg(
    Avg_Purchase_Tot = pd.NamedAgg(column = 'Price', aggfunc= lambda x: round(mean(x),2))
)

#final summary
purchase_analysis_gender = purchase_analysis.join(purchase_analysis2)

#rename columns
purchase_analysis_gender.columns = ['Number of Purchases', 'Total Revenue', 'Average Price', 'Avg Total Purchase Per Person']

#format columns
purchase_analysis_gender['Total Revenue'] = purchase_analysis_gender['Total Revenue'].map("${:,.2f}".format)
purchase_analysis_gender['Average Price'] = purchase_analysis_gender['Average Price'].map("${:,.2f}".format)
purchase_analysis_gender['Avg Total Purchase Per Person'] = purchase_analysis_gender['Avg Total Purchase Per Person'].map("${:,.2f}".format)
purchase_analysis_gender

,Number of Purchases,Total Revenue,Average Price,Avg Total Purchase Per Person
Gender,,,,
Female,113,$361.94,$3.11,$4.47
Male,652,"$1,967.64",$3.04,$4.07
Other / Non-Disclosed,15,$50.19,$3.31,$4.56


# Age Demographics

In [74]:
#select buckets for age grouping
bins = [0, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50]
labels = ["<10", "10-14", "15-18", "19-22", "23-26", "27-30", "31-34", "35-38", "39-42", "43-46", "47-50"]
fantasy_df['Age Group'] = pd.cut(fantasy_df['Age'], bins, labels=labels)


age_demo = fantasy_df.groupby('Age Group').agg({'Item ID': 'count','Price': ['sum', lambda x: round(x.unique().mean(),2)]})
age_demo.columns = age_demo.columns.droplevel(level=1)
age_demo['Price'].fillna(0)

#rename columns
age_demo.columns = ['Number of Purchases', 'Total Revenue', 'Average Price']
age_demo['Average Price'] = age_demo['Average Price'].fillna(0)

age_demo

,Number of Purchases,Total Revenue,Average Price
Age Group,,,
<10,32,108.96,3.41
10-14,19,50.95,2.77
15-18,113,342.91,2.99
19-22,254,771.89,3.03
23-26,207,634.24,3.12
27-30,63,181.23,2.88
31-34,38,103.68,2.75
35-38,35,124.35,3.50
39-42,15,50.50,3.37


# Top Spenders